# Versal xcvp1552 — HLS Kernel → xclbin Demo

Write an HLS C++ kernel inline, compile it with `v++` against the custom `custom_platform.xpfm`,
and produce a `.xclbin` ready to deploy on hardware.

**Before launching this notebook, source the tools:**
```bash
source /tools/Xilinx/Vitis/2024.2/settings64.sh
source /opt/xilinx/xrt/setup.sh
cd step6_vp1552/
jupyter lab
```

---
## 0. Environment Setup 
- The `.xpfm` was built by step3. We auto-detect it relative to this notebook's working directory, or you can override with the `VERSAL_XPFM` environment variable.

In [1]:
import shutil, os, pathlib

vpp = shutil.which("v++")
assert vpp, "v++ not found — source /tools/Xilinx/Vitis/2024.2/settings64.sh first!"
print(f"v++ : {vpp}")
!v++ --version 2>&1 | head -3

# Auto-detect: assumes Jupyter was launched from step6_vp1552/
_here = pathlib.Path.cwd()
_default_xpfm = (
    _here.parent
    / "step3_vp1552/ws/custom_platform/export/custom_platform/custom_platform.xpfm"
)

PLATFORM = os.environ.get("VERSAL_XPFM", str(_default_xpfm))

assert os.path.exists(PLATFORM), (
    f"Platform not found: {PLATFORM}\n"
    "Set VERSAL_XPFM env var or edit PLATFORM above."
)
print(f"Platform: {PLATFORM}")

v++ : /tools/Xilinx/Vitis/2024.2/bin/v++

****** v++ v2024.2 (64-bit)
  **** SW Build 5239630 on 2024-11-10-11:19:46
Platform: /home/synthara/VersalPrjs/felix/felix-xpfm-project/step3_vp1552/ws/custom_platform/export/custom_platform/custom_platform.xpfm


---
## 1. Initialise VitisKernel Builder

In [2]:
from vitis_build import VitisKernel

vk = VitisKernel(platform=PLATFORM)

Platform     : /home/synthara/VersalPrjs/felix/felix-xpfm-project/step3_vp1552/ws/custom_platform/export/custom_platform/custom_platform.xpfm
v++          : /tools/Xilinx/Vitis/2024.2/bin/v++
Build dir    : /home/synthara/VersalPrjs/felix/felix-xpfm-project/step6_vp1552/vitis_builds
rootfs       : /home/synthara/VersalPrjs/felix/felix-xpfm-project/step2_vp1552/my_foe_flx/images/linux/rootfs.ext4
kernel_image : /home/synthara/VersalPrjs/felix/felix-xpfm-project/step2_vp1552/my_foe_flx/images/linux/Image


---
## 2. Define the HLS Kernel

A simple vector-add kernel. Key points for Vitis HLS 2024.2 on Versal:

In [4]:
vadd_src = """
extern "C" {

void vadd(const int* a, const int* b, int* c, int n) {
    #pragma HLS INTERFACE m_axi port=a bundle=gmem0
    #pragma HLS INTERFACE m_axi port=b bundle=gmem1
    #pragma HLS INTERFACE m_axi port=c bundle=gmem2
    #pragma HLS INTERFACE s_axilite port=return

    for (int i = 0; i < n; i++) {
        #pragma HLS PIPELINE II=1
        c[i] = a[i] + b[i];
    }
}

}
"""

print("Kernel source ready: vadd (vector addition, 3 x m_axi bundles)")

Kernel source ready: vadd (vector addition, 3 x m_axi bundles)


---
## 3. Compile to .xclbin

| Target   | Use                                      |
|----------|------------------------------------------|
| `hw_emu` | Functional verification (QEMU + RTL sim) |
| `hw`     | Real hardware deployment                 |

Use `hw_emu` for the demo.

In [5]:
xclbin_path = vk.build(
    source=vadd_src,
    kernel_name="vadd",
    target="hw_emu",   # change to "hw" for real hardware
)

[1/3] v++ -c  (hw_emu) vadd.cpp -> .xo ...
       Compile done (42s)
[2/3] v++ -l  (hw_emu) .xo -> .xsa ...
  >> Check VPL, containing 2 checks, has run: 0 errors
  >> WARNING: [v++ 60-1455] Debuggable symbols are not generated successfully, clean up /home/synthara/VersalPrjs/felix/felix-xpfm-project/step6_vp1552/_x/link/int/consolidated.cf
  >> WARNING: Skipping CLOCK_FREQ_TOPOLOGY section for count size is zero.
  >> WARNING: Section 'CLOCK_FREQ_TOPOLOGY' content is empty.  No data in the given JSON file.
  >> Check POST-VPL, containing 1 checks, has run: 0 errors
  >> WARNING: [v++ 60-1629] No sd_card image will be generated for hardware emulation. Parameter compiler.addOutputTypes will not be honored.
       Link done (278s)
[3/3] v++ -p  (hw_emu) .xsa -> .xclbin ...
  >> WARNING: [v++ 82-10536] Platform doesn't contain boot mode
Build complete in 5.2 min
  xclbin : /home/synthara/VersalPrjs/felix/felix-xpfm-project/step6_vp1552/vitis_builds/vadd_hw_emu/vadd.xclbin (18116 KB)
  log

---
## 4. Inspect the Output

In [6]:
size_mb = os.path.getsize(xclbin_path) / (1024 * 1024)
print(f"xclbin : {xclbin_path}")
print(f"size   : {size_mb:.2f} MB")

build_dir = os.path.dirname(xclbin_path)
print(f"\nBuild artifacts in {build_dir}/")

# Dump xclbin metadata if xclbinutil is available
if shutil.which("xclbinutil"):
    !xclbinutil --input {xclbin_path} --info 2>/dev/null | head -100
else:
    print("xclbinutil not on PATH — skipping metadata dump")

xclbin : /home/synthara/VersalPrjs/felix/felix-xpfm-project/step6_vp1552/vitis_builds/vadd_hw_emu/vadd.xclbin
size   : 17.69 MB

Build artifacts in /home/synthara/VersalPrjs/felix/felix-xpfm-project/step6_vp1552/vitis_builds/vadd_hw_emu/
XRT Build Version: 2.18.179 (2024.2)
       Build Date: 2024-11-05 13:57:47
          Hash ID: 3ade2e671e5ab463400813fc2846c57edf82bb10
------------------------------------------------------------------------------
         be done in memory with the exception of the '--dump-section' command.
------------------------------------------------------------------------------
Reading xclbin file into memory.  File: /home/synthara/VersalPrjs/felix/felix-xpfm-project/step6_vp1552/vitis_builds/vadd_hw_emu/vadd.xclbin

XRT Build Version: 2.18.179 (2024.2)
       Build Date: 2024-11-05 13:57:47
          Hash ID: 3ade2e671e5ab463400813fc2846c57edf82bb10
xclbin Information
------------------
   Generated by:           v++ (2024.2) on 2024-11-10-11:19:46
   Version

---
## 5. Deploy to Board

Copy the `.xclbin` to the booted Versal board via SCP.
The kernel runs on-device with XRT (no host-side compilation needed for the demo).

In [ ]:
BOARD_IP   = "192.168.1.100"   # <<< edit to match your board
BOARD_USER = "root"
REMOTE_DIR = "/lib/firmware/xilinx"

deploy_cmd = f"scp {xclbin_path} {BOARD_USER}@{BOARD_IP}:{REMOTE_DIR}/"
print(f"Run to deploy:\n  {deploy_cmd}")

# Uncomment when board is connected:
# !{deploy_cmd}